# Streamlit Dashboard Code

In [ ]:
import streamlit as st

st.title('The Critics Critique App')
st.header('Introduction')
st.subheader('Steps')
st.text('This is an example')
st.latex('y = x²')
st.code('if a == 1:\n    print(a)', language='python')
st.markdown("This is **text** with markdown")